In [1]:
import numpy as np
import pandas as pd

import sklearn

import tensorflow as tf
from tensorflow import keras
from keras import layers, models, optimizers
from keras.regularizers import L1L2

import datawig

import os
import time
import gc


Using TensorFlow backend.


In [81]:
train_X = pd.read_csv('./train_X.csv')
test_X = pd.read_csv('./test_X.csv')

train_y = pd.read_csv('./train_y.csv')

sub = pd.read_csv('./data/sample_submission.csv')

In [82]:
cc = train_X.columns.sort_values()

rain_X = train_X[cc]
test_X = test_X[cc]

In [83]:
target = (train_y - np.mean(train_y))/np.std(train_y)

In [60]:
n_layers = 256
n_node = 4

inputs = layers.Input(shape =(train_X.shape[1], ))

x = layers.Dense(n_layers)(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for i in range(n_node):
    x = layers.Dense(n_layers//(2**(i+1)), kernel_regularizer=L1L2(l2=0.001), kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

outputs = layers.Dense(1, activation='linear')(x)

m = models.Model(inputs, outputs)

m.compile(optimizer = 'adam',
            loss = 'mse')

In [61]:
m.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 275)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 256)               70656     
_________________________________________________________________
batch_normalization_29 (Batc (None, 256)               1024      
_________________________________________________________________
activation_29 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 128)               32896     
_________________________________________________________________
batch_normalization_30 (Batc (None, 128)               512       
_________________________________________________________________
activation_30 (Activation)   (None, 128)               0   

In [62]:
es = keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)

In [63]:
m.fit(train_X, target, 
      epochs = 1000,
     validation_split=0.3,
     callbacks = [es])

Train on 1021 samples, validate on 439 samples
Epoch 1/1000
1021/1021 [==============================] - 1s 1ms/step - loss: 1.0960 - val_loss: 0.9474
Epoch 2/1000
1021/1021 [==============================] - 0s 307us/step - loss: 0.7040 - val_loss: 0.8445
Epoch 3/1000
1021/1021 [==============================] - 0s 311us/step - loss: 0.6514 - val_loss: 0.7942
Epoch 4/1000
1021/1021 [==============================] - 0s 312us/step - loss: 0.6284 - val_loss: 0.7545
Epoch 5/1000
1021/1021 [==============================] - 0s 308us/step - loss: 0.6212 - val_loss: 0.7578
Epoch 6/1000
1021/1021 [==============================] - 0s 310us/step - loss: 0.5914 - val_loss: 0.6935
Epoch 7/1000
1021/1021 [==============================] - 0s 305us/step - loss: 0.5677 - val_loss: 0.6928
Epoch 8/1000
1021/1021 [==============================] - 0s 303us/step - loss: 0.5390 - val_loss: 0.6807
Epoch 9/1000
1021/1021 [==============================] - 0s 303us/step - loss: 0.5480 - val_loss: 0.6556
E

1021/1021 [==============================] - 0s 320us/step - loss: 0.1324 - val_loss: 0.3458
Epoch 78/1000
1021/1021 [==============================] - 0s 310us/step - loss: 0.1383 - val_loss: 0.3576
Epoch 79/1000
1021/1021 [==============================] - 0s 310us/step - loss: 0.1371 - val_loss: 0.2999
Epoch 80/1000
1021/1021 [==============================] - 0s 312us/step - loss: 0.1351 - val_loss: 0.3764
Epoch 81/1000
1021/1021 [==============================] - 0s 319us/step - loss: 0.1330 - val_loss: 0.3667
Epoch 82/1000
1021/1021 [==============================] - 0s 312us/step - loss: 0.1175 - val_loss: 0.3526
Epoch 83/1000
1021/1021 [==============================] - 0s 315us/step - loss: 0.1270 - val_loss: 0.3438
Epoch 84/1000
1021/1021 [==============================] - 0s 308us/step - loss: 0.1255 - val_loss: 0.3365
Epoch 85/1000
1021/1021 [==============================] - 0s 313us/step - loss: 0.1198 - val_loss: 0.3932
Epoch 86/1000
1021/1021 [==========================

1021/1021 [==============================] - 0s 308us/step - loss: 0.0550 - val_loss: 0.2551
Epoch 154/1000
1021/1021 [==============================] - 0s 305us/step - loss: 0.0611 - val_loss: 0.2728
Epoch 155/1000
1021/1021 [==============================] - 0s 309us/step - loss: 0.0629 - val_loss: 0.2763
Epoch 156/1000
1021/1021 [==============================] - 0s 316us/step - loss: 0.0742 - val_loss: 0.2608
Epoch 157/1000
1021/1021 [==============================] - 0s 312us/step - loss: 0.0675 - val_loss: 0.2670
Epoch 158/1000
1021/1021 [==============================] - 0s 308us/step - loss: 0.0642 - val_loss: 0.3029
Epoch 159/1000
1021/1021 [==============================] - 0s 306us/step - loss: 0.0736 - val_loss: 0.2463
Epoch 160/1000
1021/1021 [==============================] - 0s 315us/step - loss: 0.0582 - val_loss: 0.2642
Epoch 161/1000
1021/1021 [==============================] - 0s 313us/step - loss: 0.0638 - val_loss: 0.2603
Epoch 162/1000
1021/1021 [=================

In [85]:
k = m.predict(test_X)

In [96]:
ans = (k * np.std(train_y)[0]) + np.mean(train_y)[0]

In [97]:
sub['SalePrice'] = ans

In [98]:
sub

,Id,SalePrice
0,1461,96352.843750
1,1462,134897.718750
2,1463,185500.015625
3,1464,196634.281250
4,1465,181895.328125
...,...,...
1454,2915,78717.148438
1455,2916,82346.812500
1456,2917,181107.375000
1457,2918,110471.937500


In [99]:
sub.to_csv('./sub/sub10.csv', index=False)